# <center>Digital Image Processing - Problem Set 6</center>

### Student Names:

### Problem 1

Write a function that performs object recognition using $k$-Nearest Neighbors ($k$-NN). You can (and are encouraged to) use the functions
you have written for previous exercises. Your function should run the following steps:

1. Binarize the images in the training set. Use an appropriate threshold when binarizing each image. You can use an adaptive thresholding technique. Perform binarization such that pixels on the object of interest are equal to 1 and background pixels are equal to 0.
2. Describe each object in the training set using Hu moments. This results in one feature vector per object.
3. Repeat the two previous steps for the all images in the training set.
4. The $k$-NN classifier predicts a category for each testing example according to the following steps:
    1. Pick an image from the testing set.
    2. Compute the distance from the feature vector associated to the testing example to all the descriptors from the training set.
    3. Select the closest $k$ training examples (that is, the $k$ training examples with shortest distance to our testing image).
    4. Find the object category associated to each of the $k$ training examples. This information is given to you in the training set.
    5. Assign a label to the testing image. The assigned label is equal to the most common label among the $k$ closes training examples.
    
5. Repeat this process for all testing examples. Compute the accuracy of the classifier in terms of the number of correctly classified testing examples over the total number of examples in the testing set.

You should test your functions with the three object classes <tt> key, chip, pen</tt>. Compare the difference in performance between a 1-NN classifier, a 3-NN classifier and a $k$-NN classifier. Summarize the performance of each classifier using confusion matrices.